In [1]:
import cv2
import pickle
import numpy as np

In [50]:
posList = []

width = 27
height = 11

try:
    with open(file="videos/CarParkPos", mode="rb") as f:
        posList = pickle.load(f)
except:
    posList = []



def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x, y))
    
    if events == cv2.EVENT_RBUTTONDOWN:
        for id, position in enumerate(posList):
            x1, y1 = position
            if x1 < x < x1 + width and y1 < y < y1 + height:
                posList.pop(id)
    
    with open(file="videos/CarParkPos", mode="wb") as f:
        pickle.dump(obj=posList, file=f)

while True:
    img = cv2.imread(filename="videos/first_frame.png")
    
    for position in posList:
        cv2.rectangle(img=img, 
                      pt1=position, 
                      pt2=(position[0] + width, position[1] + height), 
                      color=(255, 0, 0), 
                      thickness=2)
    
    
    cv2.imshow(winname="img", mat=img)
    cv2.setMouseCallback(window_name="img", on_mouse=mouseClick)
    if cv2.waitKey(delay=1) == ord("q"): break
video.release()
cv2.destroyAllWindows()

In [64]:
video = cv2.VideoCapture(filename="videos/carParkVideo.mp4")

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

#writer = cv2.VideoWriter(filename="videos/carPark.mp4",
#                         fourcc=cv2.VideoWriter_fourcc(*"DIVX"),
#                         fps=25,
#                         frameSize=(width, height))
width, height = 27, 15

with open(file="videos/CarParkPos", mode="rb") as f:
    posList = pickle.load(f)
    
    
    
def checkParkSpace(imgg):
    spaceCounter = 0
    
    for position in posList:
        x, y = position
        
        img_crop = imgg[y: y+height, x: x+width]
        count = cv2.countNonZero(src=img_crop)
        
        
        if count < 150:
            color = (0, 255, 0)
            thickness = 2
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2
        
        cv2.rectangle(img=img,
                      pt1=position,
                      pt2=(position[0]+width, position[1]+height),
                      color=color,
                      thickness=thickness)
        cv2.putText(img=img,
                    text=str(count),
                    org=(x, y+height-2),
                    fontFace=cv2.FONT_HERSHEY_PLAIN,
                    fontScale=1,
                    color=color,
                    thickness=1)
    cv2.putText(img=img,
                 text=f"Free: {spaceCounter}/{len(posList)}",
                 org=(15, 15),
                 fontFace=cv2.FONT_HERSHEY_PLAIN,
                 fontScale=1,
                 color=(0, 255, 0),
                 thickness=2)


while True:
    retval, img = video.read()
    if not retval: break
    
    imgGray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(src=imgGray, ksize=(3, 3), sigmaX=1)
    imgThreshold = cv2.adaptiveThreshold(src=imgBlur, 
                                         maxValue=255, 
                                         adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         thresholdType=cv2.THRESH_BINARY_INV,
                                         blockSize=25,
                                         C=16)
    imgMedian = cv2.medianBlur(src=imgThreshold, ksize=5)
    imgDilate = cv2.morphologyEx(src=imgMedian, 
                                 op=cv2.MORPH_DILATE, 
                                 kernel=np.ones((3, 3), np.uint8),
                                 iterations=1)
    checkParkSpace(imgg=imgDilate)
    
    #writer.write(image=img)
    cv2.imshow(winname="Video", mat=img)
    if cv2.waitKey(delay=170) == ord("q"): break

#writer.release()
video.release()
cv2.destroyAllWindows()